To dos: 
    
    1) EDA on test see if it's similar with train
    
    2) FE add time compmenets according to different region e.g. day of week etc
    
    3) test impression list length

Notes:

    * total train shape: (15,932,992, 12)
    * no action type and reference is nan in train

In [1]:
import pandas as pd
import numpy as np
from utils import load_data

In [30]:
%%time
train = load_data('train', nrows=int(1e6))
# train = load_data('train', nrows=None)

CPU times: user 1.21 s, sys: 91.9 ms, total: 1.31 s
Wall time: 1.3 s


total train shape: (15,932,992, 12)

In [4]:
train.shape

(1000000, 12)

In [9]:
ref_act_size = train.groupby(['reference', 'action_type']).size()
dist = ref_act_size.groupby(level='action_type').value_counts()

In [15]:
dist

action_type                
change of sort order  241          1
                      436          1
                      623          1
                      636          1
                      736          1
                      2725         1
                      4948         1
                      14589        1
clickout item         1        36790
                      2        10392
                      3         4092
                      4         1935
                      5         1050
                      6          566
                      7          371
                      8          242
                      9          166
                      10         121
                      11          97
                      12          60
                      14          47
                      13          36
                      15          36
                      16          24
                      18          21
                      17          15
          

In [21]:
dist.iloc[dist.index.get_level_values('action_type') == 'clickout item']

action_type      
clickout item  1     36790
               2     10392
               3      4092
               4      1935
               5      1050
               6       566
               7       371
               8       242
               9       166
               10      121
               11       97
               12       60
               14       47
               13       36
               15       36
               16       24
               18       21
               17       15
               19       13
               20       11
               21        8
               22        6
               23        5
               24        5
               26        4
               28        4
               32        3
               27        2
               41        2
               25        1
               29        1
               31        1
               34        1
               35        1
               36        1
               37        1
          

In [33]:
# clickout_vc = train[train.action_type=='clickout item'].groupby('reference').size().value_counts()

In [141]:
ref_act = train[['reference', 'action_type']].copy()
ref_act = ref_act[~ref_act.reference.str.contains('[a-zA-Z]')].reset_index(drop=True)
unique_actions = ref_act['action_type'].unique()
action2natural = {v: k for k, v in enumerate(unique_actions)}
action_names = list(action2natural.keys())
actions = ref_act['action_type'].map(action2natural).values

actions_ohe = pd.DataFrame(np.eye(len(unique_actions), dtype=int)[actions], columns=action_names)

In [142]:
action2natural

{'interaction item image': 0,
 'clickout item': 1,
 'interaction item info': 2,
 'interaction item deals': 3,
 'interaction item rating': 4,
 'search for item': 5}

In [143]:
ref_act = pd.concat([ref_act, actions_ohe], axis=1)

In [144]:
M = [2]*6
for k, c in enumerate(action_names):
    print(c)
    # Compute the global mean
    mu = ref_act[c].mean()

    # Compute the number of values and the mean of each group
    agg = ref_act.groupby('reference')[c].agg(['count', 'mean'])
    counts = agg['count']
    mus = agg['mean']
    # Compute the "smoothed" means
    smoothed = (counts*mus + M[k]*mu)/(counts + M[k])
    ref_act[c] = ref_act['reference'].map(smoothed)
    #     # Replace each value by the according smoothed mean
#     return df[by].map(smooth)

interaction item image
clickout item
interaction item info
interaction item deals
interaction item rating
search for item


In [145]:
ref_act.head()

,reference,action_type,interaction item image,clickout item,interaction item info,interaction item deals,interaction item rating,search for item
0,666856,interaction item image,0.865757,0.122273,0.003997,0.002719,0.003063,0.002191
1,666856,interaction item image,0.865757,0.122273,0.003997,0.002719,0.003063,0.002191
2,666856,interaction item image,0.865757,0.122273,0.003997,0.002719,0.003063,0.002191
3,109038,interaction item image,0.610732,0.384280,0.001665,0.001133,0.001276,0.000913
4,666856,interaction item image,0.865757,0.122273,0.003997,0.002719,0.003063,0.002191


clicked vs viewed

In [146]:
ref_imp = train.loc[train.action_type=='clickout item', ['reference', 'impressions']].reset_index(drop=True)
ref_imp = ref_imp[~ref_imp.reference.str.contains('[a-zA-Z]')].reset_index(drop=True)
# ref_imp = ref_imp[ref_imp]

In [147]:
ref_imp['impressions'] = ref_imp['impressions'].str.split('|')

In [148]:
%%time
ref_imp['impressions'] = ref_imp.apply(lambda row: list(set(row.impressions) - set([row.reference])), axis=1)

CPU times: user 2.94 s, sys: 3.58 ms, total: 2.94 s
Wall time: 2.94 s


In [149]:
ref_imp.head()

,reference,impressions
0,109038,"[55088, 1253714, 666856, 3909420, 1088584, 104..."
1,1257342,"[55088, 1332971, 2297972, 2050977, 129343, 116..."
2,2795374,"[1258068, 8118684, 3143352, 6652864, 107162, 3..."
3,1032816,"[81657, 47687, 46363, 106471, 18425, 18417, 98..."
4,1032816,"[81657, 47687, 46363, 106471, 18425, 18417, 98..."


In [150]:
imps = ref_imp.impressions.values
imps = [j for i in imps for j in i]
click_imp = pd.concat([pd.Series([0]*len(imps), index=imps), 
                       pd.Series([1]*len(ref_imp), index=ref_imp.reference.values)])
click_imp.index.name = 'item_id'
click_imp = pd.DataFrame(click_imp, columns=['clicked']).reset_index()

In [151]:
mu = click_imp['clicked'].mean()
m = 2
agg = click_imp.groupby('item_id')['clicked'].agg(['count', 'mean'])
count = agg['count']
mus = agg['mean']
smoothed = (count*mus + m*mu)/(count + m)
click_imp['clicked'] = click_imp['item_id'].map(smoothed)

In [152]:
click_imp.head()

,item_id,clicked
0,55088,0.009667
1,1253714,0.012429
2,666856,0.271751
3,3909420,0.006692
4,1088584,0.098818
